In [ ]:
from flask import Flask, jsonify
import os
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
import gspread
from google.oauth2.service_account import Credentials

# Configuração do Flask
app = Flask(__name__)

# Caminhos e arquivos
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) + os.sep
SCALER_PATH = os.path.join(BASE_DIR, 'scaler.pkl')
ENCODER_PATH = os.path.join(BASE_DIR, 'encoder.pkl')
FEATURE_COLS_PATH = os.path.join(BASE_DIR, 'feature_cols.csv')
MODEL_PATH = os.path.join(BASE_DIR, 'meu_modelo_TEA.h5')
SERVICE_ACCOUNT_FILE = os.path.join(BASE_DIR, 'service_account.json')
SHEET_ID = 'COLOQUE_AQUI_O_ID_DA_PLANILHA'

# Carregar modelo e pré-processadores
scaler = joblib.load(SCALER_PATH)
encoder = joblib.load(ENCODER_PATH)
feature_cols = pd.read_csv(FEATURE_COLS_PATH, header=None).iloc[:,0].tolist()
model = load_model(MODEL_PATH)

# Função de transformação
def transform_new(df_new, scaler, encoder, feature_cols):
    df = df_new.copy()
    yes_no_cols = ['Col01','Col02','Col03','Col04','Col05','Col06','Col07','Col08','Col09','Col10','Col14','Col16']
    for col in yes_no_cols:
        if col in df.columns:
            df[col] = df[col].map({'Sim': 1, 'Não': 0}).astype('int8')
    df.replace("?", np.nan, inplace=True)
    if 'Col12' in df.columns:
        df['Col12'] = df['Col12'].map({'Masculino': 0, 'Feminino': 1}).astype('int8')
    if 'Col11' in df.columns:
        df[['Col11']] = scaler.transform(df[['Col11']].astype(float))
    if df.shape[1] >= 10 and 'Col18' not in df.columns:
        df['Col18'] = df.iloc[:, :10].apply(pd.to_numeric, errors='coerce').sum(axis=1)
    categorical_cols = ['Col13', 'Col15', 'Col17']
    for c in categorical_cols:
        if c in df.columns:
            df[c] = df[c].fillna('missing')
    present_cats = [c for c in categorical_cols if c in df.columns]
    if present_cats:
        encoded = encoder.transform(df[present_cats])
        feature_names = encoder.get_feature_names_out(present_cats)
        df = df.drop(columns=present_cats).reset_index(drop=True)
        df = pd.concat([df, pd.DataFrame(encoded, columns=feature_names)], axis=1)
    df = df.reindex(columns=feature_cols, fill_value=0)
    return df

# Função para buscar os dados mais recentes do Google Sheets
def get_latest_form_data():
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets'])
    gc = gspread.authorize(creds)
    sh = gc.open_by_key('1p5yluLZ2vO41cAfYwBBpP4tiX14XMgpy4nBvfcJhWX0')
    worksheet = sh.sheet1
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    latest_row = df.tail(1)  # última resposta
    return latest_row

@app.route('/predict', methods=['GET'])
def predict():
    try:
        df_cliente = get_latest_form_data()
        df_cliente_transformado = transform_new(df_cliente, scaler, encoder, feature_cols)
        novo_dado = df_cliente_transformado.values.reshape(1, -1)
        predicoes = model.predict(novo_dado)
        prob = float(predicoes[0][0])
        label = 'TEA' if prob > 0.5 else 'Não-TEA'
        return jsonify({
            'probability': prob,
            'label': label
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=int(os.environ.get('PORT', 5000)))